In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv")
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data = pd.DataFrame.rename(data, columns={'message to examine':'text'})

In [ ]:
X = data['text']
y = data['label (depression result)']

In [ ]:
X

In [ ]:
y

In [ ]:
message = data.copy()

In [ ]:
message

In [ ]:
import nltk
from nltk.corpus import stopwords
import re

In [ ]:
nltk.download('stopwords')

In [ ]:
 message['text'][1]

In [ ]:
# data Preprocessing
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0, len(message)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', message['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word)for word in review if not word in stopwords.words('english')]
    review =  ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

In [ ]:
voc_size = 10000

In [ ]:
onehot_repr = [one_hot(words, voc_size)for words in corpus]
onehot_repr

In [ ]:
sent_len=20
word_embedding = pad_sequences(onehot_repr, sent_len)

In [ ]:
word_embedding

In [ ]:
feature_len = 40

In [ ]:
feature_size = 40
model = Sequential()

model.add(Embedding(voc_size, feature_size, input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
X_new = np.array(word_embedding)
y_new = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.3)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

In [ ]:
y_pred = model.predict_classes(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
accuracy = accuracy_score(y_pred, y_test)
accuracy

In [ ]:
report = classification_report(y_pred, y_test)
print(report)

In [ ]:
matrix = confusion_matrix(y_pred, y_test)
matrix